In [1]:
# Importing libraries
import requests
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import string
import re

In [87]:
# Importing more libraries
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
%matplotlib inline

In [3]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [4]:
# Xpath
part_1 = "//div[@class='lister-list']/div[@class='lister-item mode-advanced']["
part_2 = "]/div[@class='lister-item-content']/h3[@class='lister-item-header']/a/@href"
# url link to imdb search results
url_1 = "http://www.imdb.com/search/title?adult=include&countries=us&languages=en&num_votes=1000,&release_date=2005,&title_type=tv_series&user_rating=1.0,10&sort=user_rating,desc&view=advanced"
url_2 = "&page="
url_3 = "&ref_=adv_nxt"

In [5]:
# Webscrapping imdb movie_id list
id_list = []
for u in range(1, 29):
    URL = (url_1 + url_2 + str(u))
    r = requests.get(URL)
    HTML = r.text
    for i in range(1, 51):
        a_tag = Selector(text=HTML).xpath(part_1+str(i)+part_2).extract()
        for a in a_tag:
            id_list.append(a[7:16])

In [6]:
len(id_list)

1379

In [7]:
id_list[570]

u'tt0460666'

In [8]:
# Function to access the tv maze app, retrieve data and also accesses the imdb page to webscrape some more information
# converts data into dataframe
def get_movie_info(entry):
    res=requests.get('http://api.tvmaze.com/lookup/shows?imdb='+entry)
    if res.status_code == 200:
        results = json.loads(res.text)
        
        try:
            name = results['name']
        except TypeError:
            name = 'NaN'

        try:
            rating =  results['rating']['average']
        except TypeError:
            rating = 'NaN'

        try:
            genres = results['genres']
        except TypeError:
            genres = 'NaN'

        try:
            network = results['network']['name']
        except TypeError:
            network = 'NaN'

        try:
            premiered = results['premiered']
        except TypeError:
            premiered = 'NaN'

        try:
            status = results['status']
        except TypeError:
            status = 'NaN'

        try:
            types = results['type']
        except TypeError:
            types = 'NaN'
        
        try:
            show_id = results['id']
        except TypeError:
            show_id = 'NaN'
            
        url = "http://www.imdb.com/title/" + entry + "/reviews?ref_=tt_urv"
        r = requests.get(url)
        HTML = r.text
        path1 = "//div[@id='tn15content']/p["
        path2 = "]/text()"
        a_tag_list = []
        for i in range(1, 11):
            try:
                a_tag = Selector(text=HTML).xpath(path1 + str(i) + path2).extract()
                a_tag_list.append(a_tag[0])
            except:
                a_tag_list.append('NaN')
           
        res2 = requests.get('http://api.tvmaze.com/shows/' + str(show_id) + '/crew')
        crew_list_i = []
        if res2.status_code == 200:
            results2 = json.loads(res2.text)
            for number in range(len(results2)):
                try:
                    crew_list_i.append(results2[number]['person']['name'])
                except:
                    pass
        crew_set = set(crew_list_i)
        crew_list = list(crew_set)
        
        res3 = requests.get('http://api.tvmaze.com/shows/' + str(show_id))
        if res3.status_code == 200:
            results3 = json.loads(res3.text)
            
            try:
                summary = results['summary']
            except:
                summary = 'NaN'
            

        
        shows.loc[len(shows)] = [name, rating, genres, network, premiered, status, types, crew_list, summary, a_tag_list] 
            
        

In [9]:
shows = pd.DataFrame(columns = ['show_name', 'rating_avg', 'genres', 'network', 'premiere_date', 'status', 'types', 'crew', 'summary', 'reviews'])

counter = 0
for entry in id_list:
    get_movie_info(entry)
    counter = counter + 1
    print counter

In [10]:
shows.head()

,show_name,rating_avg,genres,network,premiere_date,status,types,crew,summary,reviews
0,Critical Role,NaN,[Adventure],NaN,2015-03-13,Running,Game Show,[],<p>Voice actor Matthew Mercer leads a group of...,[\n9 voice actors get together and play Dungeo...
1,Game of Thrones,9.3,"[Drama, Adventure, Fantasy]",HBO,2011-04-17,Running,Scripted,"[Katie Weiland, Lisa Byrne, George R. R. Marti...",<p>Based on the bestselling book series <i>A S...,[\nDo not believe any of those negative review...
2,Breaking Bad,9.3,"[Drama, Crime, Thriller]",AMC,2008-01-20,Ended,Scripted,"[Mark Johnson, Gina Scheerer, Andrew Ortner, V...",<p><b>Breaking Bad</b> follows protagonist Wal...,[\nI probably haven't been hooked to a TV show...
3,Rick and Morty,9.4,"[Comedy, Adventure, Science-Fiction]",Adult Swim,2013-12-02,Running,Animation,"[Joe Russo II, Dan Guterman, Ryan Ridley, Tom ...","<p>Rick is a mentally gifted, but sociopathic ...",[\nThis show is honestly great. It is easily t...
4,Sherlock,9.2,"[Drama, Crime, Mystery]",BBC One,2010-07-25,To Be Determined,Scripted,"[Charlotte Ashby, Jeremy Child, Suzie Lavelle,...",<p>Sherlock Holmes and Dr. John Watson's adven...,"[\n..with delight. A brilliantly written, well..."


In [11]:
shows.to_pickle('shows.pkl')

In [12]:
df = shows.copy()

In [13]:
# data cleaning
def remove_html(string):
    return re.sub(r'<.+?>', '', string)

In [14]:
# stripping html tag characters
df['summary'] = df['summary'].apply(lambda x: remove_html(x))

In [15]:
df.head()

,show_name,rating_avg,genres,network,premiere_date,status,types,crew,summary,reviews
0,Critical Role,NaN,[Adventure],NaN,2015-03-13,Running,Game Show,[],Voice actor Matthew Mercer leads a group of fe...,[\n9 voice actors get together and play Dungeo...
1,Game of Thrones,9.3,"[Drama, Adventure, Fantasy]",HBO,2011-04-17,Running,Scripted,"[Katie Weiland, Lisa Byrne, George R. R. Marti...",Based on the bestselling book series A Song of...,[\nDo not believe any of those negative review...
2,Breaking Bad,9.3,"[Drama, Crime, Thriller]",AMC,2008-01-20,Ended,Scripted,"[Mark Johnson, Gina Scheerer, Andrew Ortner, V...","Breaking Bad follows protagonist Walter White,...",[\nI probably haven't been hooked to a TV show...
3,Rick and Morty,9.4,"[Comedy, Adventure, Science-Fiction]",Adult Swim,2013-12-02,Running,Animation,"[Joe Russo II, Dan Guterman, Ryan Ridley, Tom ...","Rick is a mentally gifted, but sociopathic and...",[\nThis show is honestly great. It is easily t...
4,Sherlock,9.2,"[Drama, Crime, Mystery]",BBC One,2010-07-25,To Be Determined,Scripted,"[Charlotte Ashby, Jeremy Child, Suzie Lavelle,...",Sherlock Holmes and Dr. John Watson's adventur...,"[\n..with delight. A brilliantly written, well..."


In [16]:
# stripping newline spaces
df['reviews'] = df.reviews.map(lambda y: [x.replace('\n','') for x in y])

In [17]:
df.shape

(1268, 10)

In [18]:
df.isnull().sum()

show_name          0
rating_avg       130
genres             0
network            0
premiere_date      0
status             0
types              0
crew               0
summary            0
reviews            0
dtype: int64

In [19]:
df.types.value_counts()

Scripted       921
Reality        146
Animation      119
Documentary     30
Talk Show       22
Variety         14
Game Show       10
News             3
Sports           3
Name: types, dtype: int64

In [20]:
# dropping rows where targe is missing
df.dropna(axis=0, inplace=True)

In [21]:
df.isnull().sum()

show_name        0
rating_avg       0
genres           0
network          0
premiere_date    0
status           0
types            0
crew             0
summary          0
reviews          0
dtype: int64

In [22]:
df.shape

(1138, 10)

In [23]:
df.head()

,show_name,rating_avg,genres,network,premiere_date,status,types,crew,summary,reviews
1,Game of Thrones,9.3,"[Drama, Adventure, Fantasy]",HBO,2011-04-17,Running,Scripted,"[Katie Weiland, Lisa Byrne, George R. R. Marti...",Based on the bestselling book series A Song of...,[Do not believe any of those negative reviews....
2,Breaking Bad,9.3,"[Drama, Crime, Thriller]",AMC,2008-01-20,Ended,Scripted,"[Mark Johnson, Gina Scheerer, Andrew Ortner, V...","Breaking Bad follows protagonist Walter White,...",[I probably haven't been hooked to a TV show l...
3,Rick and Morty,9.4,"[Comedy, Adventure, Science-Fiction]",Adult Swim,2013-12-02,Running,Animation,"[Joe Russo II, Dan Guterman, Ryan Ridley, Tom ...","Rick is a mentally gifted, but sociopathic and...",[This show is honestly great. It is easily the...
4,Sherlock,9.2,"[Drama, Crime, Mystery]",BBC One,2010-07-25,To Be Determined,Scripted,"[Charlotte Ashby, Jeremy Child, Suzie Lavelle,...",Sherlock Holmes and Dr. John Watson's adventur...,"[..with delight. A brilliantly written, well a..."
5,Last Week Tonight with John Oliver,9.0,[Comedy],HBO,2014-04-27,Running,Talk Show,"[Tim Carvell, Jon Thoday]","On Last Week Tonight with John Oliver, John Ol...",[Loved John Oliver on Community and love him o...


In [24]:
df.genres


1                [Drama, Adventure, Fantasy]
2                   [Drama, Crime, Thriller]
3       [Comedy, Adventure, Science-Fiction]
4                    [Drama, Crime, Mystery]
5                                   [Comedy]
6                                   [Action]
7          [Drama, Science-Fiction, Western]
8                          [Drama, Thriller]
9          [Drama, Crime, Thriller, Mystery]
10                            [Drama, Crime]
11                [Science-Fiction, History]
12                                        []
13                            [Drama, Crime]
14         [Drama, Science-Fiction, Mystery]
15        [Adventure, Mystery, Supernatural]
16                                        []
17                    [Drama, Action, Crime]
18                            [Drama, Crime]
19      [Adventure, Family, Science-Fiction]
22                                  [Comedy]
23                           [Drama, Comedy]
24                                  [Comedy]
25        

In [26]:
# unpacking list
df['genres'] = df['genres'].apply(lambda x: '|'.join(map(str, x)))

In [27]:
df.genres.value_counts()

Comedy                                       197
                                              96
Drama                                         45
Drama|Crime                                   44
Drama|Comedy                                  43
Comedy|Family                                 42
Drama|Action|Crime                            29
Drama|Romance                                 24
Comedy|Romance                                21
Action|Adventure|Science-Fiction              17
Drama|Crime|Thriller                          15
Drama|Science-Fiction|Thriller                15
Drama|Action|Science-Fiction                  14
Drama|Comedy|Romance                          13
Drama|Comedy|Crime                            13
Drama|Crime|Mystery                           13
Comedy|Children                               11
Drama|Thriller                                 9
Drama|Action|Adventure                         9
Food                                           8
Action|Science-Ficti

In [28]:
# unpacking list
df['crew'] = df['crew'].apply(lambda x: '|'.join(map(str, x)))

In [29]:
df.head()

,show_name,rating_avg,genres,network,premiere_date,status,types,crew,summary,reviews
1,Game of Thrones,9.3,Drama|Adventure|Fantasy,HBO,2011-04-17,Running,Scripted,Katie Weiland|Lisa Byrne|George R. R. Martin|A...,Based on the bestselling book series A Song of...,[Do not believe any of those negative reviews....
2,Breaking Bad,9.3,Drama|Crime|Thriller,AMC,2008-01-20,Ended,Scripted,Mark Johnson|Gina Scheerer|Andrew Ortner|Vince...,"Breaking Bad follows protagonist Walter White,...",[I probably haven't been hooked to a TV show l...
3,Rick and Morty,9.4,Comedy|Adventure|Science-Fiction,Adult Swim,2013-12-02,Running,Animation,Joe Russo II|Dan Guterman|Ryan Ridley|Tom Kauf...,"Rick is a mentally gifted, but sociopathic and...",[This show is honestly great. It is easily the...
4,Sherlock,9.2,Drama|Crime|Mystery,BBC One,2010-07-25,To Be Determined,Scripted,Charlotte Ashby|Jeremy Child|Suzie Lavelle|Nev...,Sherlock Holmes and Dr. John Watson's adventur...,"[..with delight. A brilliantly written, well a..."
5,Last Week Tonight with John Oliver,9.0,Comedy,HBO,2014-04-27,Running,Talk Show,Tim Carvell|Jon Thoday,"On Last Week Tonight with John Oliver, John Ol...",[Loved John Oliver on Community and love him o...


In [30]:
# unpacking list
df['reviews'] = df['reviews'].apply(lambda x: ', '.join(map(str, x)))

In [31]:
df.head()

,show_name,rating_avg,genres,network,premiere_date,status,types,crew,summary,reviews
1,Game of Thrones,9.3,Drama|Adventure|Fantasy,HBO,2011-04-17,Running,Scripted,Katie Weiland|Lisa Byrne|George R. R. Martin|A...,Based on the bestselling book series A Song of...,Do not believe any of those negative reviews. ...
2,Breaking Bad,9.3,Drama|Crime|Thriller,AMC,2008-01-20,Ended,Scripted,Mark Johnson|Gina Scheerer|Andrew Ortner|Vince...,"Breaking Bad follows protagonist Walter White,...",I probably haven't been hooked to a TV show li...
3,Rick and Morty,9.4,Comedy|Adventure|Science-Fiction,Adult Swim,2013-12-02,Running,Animation,Joe Russo II|Dan Guterman|Ryan Ridley|Tom Kauf...,"Rick is a mentally gifted, but sociopathic and...",This show is honestly great. It is easily the ...
4,Sherlock,9.2,Drama|Crime|Mystery,BBC One,2010-07-25,To Be Determined,Scripted,Charlotte Ashby|Jeremy Child|Suzie Lavelle|Nev...,Sherlock Holmes and Dr. John Watson's adventur...,"..with delight. A brilliantly written, well ac..."
5,Last Week Tonight with John Oliver,9.0,Comedy,HBO,2014-04-27,Running,Talk Show,Tim Carvell|Jon Thoday,"On Last Week Tonight with John Oliver, John Ol...",Loved John Oliver on Community and love him on...


In [32]:
# removing commas
df['summary'] = df['summary'].str.replace(',', '')

In [33]:
# removing commas
df['reviews'] = df['reviews'].str.replace(',', '')

In [34]:
df.shape

(1138, 10)

In [35]:
df.dtypes

show_name         object
rating_avg       float64
genres            object
network           object
premiere_date     object
status            object
types             object
crew              object
summary           object
reviews           object
dtype: object

In [36]:
rating_median = df.rating_avg.median()
rating_median

7.8

In [37]:
# converting target to binary class
y = df['rating_avg'].apply(lambda x: 1 if x > rating_median else 0)

In [38]:
y.value_counts()

0    614
1    524
Name: rating_avg, dtype: int64

In [39]:
# baseline accuracy
baseline = 614 / (520 + 614.)
baseline

0.5414462081128748

In [40]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(df[['show_name', 'genres', 'types', 'crew', 'summary']], y, test_size = 0.3, random_state=42)




In [41]:
X_train

,show_name,genres,types,crew,summary
382,The Brink,Comedy|War,Scripted,Kim Benabib|Roberto Benabib,The Brink focuses on a geopolitical crisis and...
977,Web Therapy,Comedy,Scripted,Diane Charles|Dan Bucatinsky|Kyle McNally|Lisa...,Fiona Wallice is a therapist with little patie...
1255,Snooki & JWoww,,Reality,,The show returns with 12 all-new episodes chro...
513,Go On,Comedy,Scripted,Matthew Perry|Aaron Miller|Jon Pollack|Liz Bri...,Following the sudden death of his wife sports ...
752,Kung Fu Panda: Legends of Awesomeness,Adventure|Children,Animation,,"""Kung Fu Panda: Legends of Awesomeness"" is bas..."
745,The Astronaut Wives Club,Drama|History,Scripted,Michael London|Lily Koppel|Josh Schwartz|Steph...,The Astronaut Wives Club tells the story of th...
932,Anger Management,Comedy,Scripted,Clay Graham|Ramon Estevez|Janae Bakken|Howard ...,In Anger Management Charlie is a non-tradition...
74,The Walking Dead,Drama|Action|Horror,Scripted,Mandy McCowan|Jessica Drake|Sharon Bialy|Greg ...,The Walking Dead tells the story of the months...
482,Big Love,Drama,Scripted,Lee Scott|Peter Friedlander|Byron Smith|R. Rus...,Bill Henrickson seems like a typical suburban ...
795,Ben 10,Action|Adventure|Science-Fiction,Animation,Duncan Rouleau|Alex Soto|Joe Kelly|Steve Seagl...,First premiering in January 2006 Ben 10 is a C...


In [42]:
y_train

382     1
977     0
1255    0
513     0
752     1
745     0
932     0
74      1
482     1
795     0
1107    0
445     0
268     1
894     0
328     1
1074    1
235     0
249     1
1127    0
1051    0
321     0
451     1
719     0
1172    0
263     0
990     0
553     0
80      0
103     1
958     1
       ..
1026    0
203     1
1045    1
736     0
538     1
1176    1
496     1
909     0
788     0
266     0
24      1
481     1
290     1
1024    1
405     0
855     0
363     1
814     1
140     1
926     0
1238    1
96      1
350     1
488     0
131     1
1124    1
1192    1
1250    0
913     0
1242    0
Name: rating_avg, Length: 796, dtype: int64

## Creating Vector DF for columns with text in training set

In [43]:
# cv_r = CountVectorizer(ngram_range=(1,3), lowercase=True, max_features=1000, stop_words='english')
# cv_r.fit(X_train['reviews'])
# words_train_r = cv_r.transform(X_train.reviews.values)
# words_train_r = pd.DataFrame(words_train_r.todense(), columns=cv_r.get_feature_names())
# words_train_r.head()

In [44]:
# words_train_r.shape

In [47]:
cv_s = CountVectorizer(ngram_range=(1,3), lowercase=True, max_features=500, stop_words='english', strip_accents='unicode')
cv_s.fit(X_train['summary'])
words_train_s = cv_s.transform(X_train.summary.values)
words_train_s = pd.DataFrame(words_train_s.todense(), columns=cv_s.get_feature_names())
words_train_s.head()

,10,12,13,abilities,action,actually,adult,adventure,adventures,age,...,working,works,world,wrong,year,year old,years,york,york city,young
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [48]:
words_train_s.shape

(796, 500)

In [49]:
# cv_c = CountVectorizer(ngram_range=(2,2), lowercase=False, max_features=1000, stop_words='english', token_pattern='\w{4,}')
# cv_c.fit(X_train['crew'])
# words_train_c = cv_c.transform(X_train.crew.values)
# words_train_c = pd.DataFrame(words_train_c.todense(), columns=cv_c.get_feature_names())
# words_train_c.head()

In [53]:
cv_c = CountVectorizer(lowercase=False, max_features=500, stop_words='english', tokenizer=(lambda x: x.split('|') ), binary=True, strip_accents='unicode')
cv_c.fit(X_train['crew'])
words_train_c = cv_c.transform(X_train.crew.values)
words_train_c = pd.DataFrame(words_train_c.todense(), columns=cv_c.get_feature_names())
words_train_c.head()

,,Aaron Kaplan,Aaron Miller,Adam Armus,Adam Bernstein,Adam Goldworm,Adam Horowitz,Adam McKay,Adam Targum,Adele Lim,...,"Victor Nelli, Jr.",Vincent Angell,Vladimir Stefoff,Wendy O'Brien,Wendy Weidman,Will Arnett,Will Bates,William Klug,Yahlin Chang,Yuri Reese
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
words_train_c.shape

(796, 1000)

In [60]:
cv_t = CountVectorizer(lowercase=False, stop_words='english', tokenizer=(lambda x: x.split('|') ), binary=True, strip_accents='unicode')
cv_t.fit(X_train['types'])
words_train_t = cv_t.transform(X_train.types.values)
words_train_t = pd.DataFrame(words_train_t.todense(), columns=cv_t.get_feature_names())
words_train_t.head()

,Animation,Documentary,Game Show,News,Reality,Scripted,Sports,Talk Show,Variety
0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,0
4,1,0,0,0,0,0,0,0,0


In [61]:
words_train_t.shape

(796, 9)

In [58]:
cv_g = CountVectorizer(lowercase=False, stop_words='english', tokenizer=(lambda x: x.split('|') ), binary=True, strip_accents='unicode')
cv_g.fit(X_train['genres'])
words_train_g = cv_g.transform(X_train.genres.values)
words_train_g = pd.DataFrame(words_train_g.todense(), columns=cv_g.get_feature_names())
words_train_g.head()

,,Action,Adult,Adventure,Anime,Children,Comedy,Crime,DIY,Drama,...,Mystery,Nature,Romance,Science-Fiction,Sports,Supernatural,Thriller,Travel,War,Western
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
words_train_g.shape

(796, 29)

In [66]:
# joing data frame together
features_train = pd.concat([words_train_c, words_train_g, words_train_s, words_train_t], axis=1)


In [67]:
features_train.head()

,,Aaron Kaplan,Aaron Miller,Adam Armus,Adam Bernstein,Adam Goldworm,Adam Horowitz,Adam McKay,Adam Targum,Adele Lim,...,young,Animation,Documentary,Game Show,News,Reality,Scripted,Sports,Talk Show,Variety
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [64]:
words_test_s = cv_s.transform(X_test.summary.values)
words_test_s = pd.DataFrame(words_test_s.todense(), columns=cv_s.get_feature_names())

words_test_c = cv_c.transform(X_test.crew.values)
words_test_c = pd.DataFrame(words_test_c.todense(), columns=cv_c.get_feature_names())

words_test_t = cv_t.transform(X_test.types.values)
words_test_t = pd.DataFrame(words_test_t.todense(), columns=cv_t.get_feature_names())

words_test_g = cv_g.transform(X_test.genres.values)
words_test_g = pd.DataFrame(words_test_g.todense(), columns=cv_g.get_feature_names())



features_test = pd.concat([words_test_c, words_test_g, words_test_s, words_test_t], axis=1)

In [65]:
features_test.head()

,,Aaron Kaplan,Aaron Miller,Adam Armus,Adam Bernstein,Adam Goldworm,Adam Horowitz,Adam McKay,Adam Targum,Adele Lim,...,young,Animation,Documentary,Game Show,News,Reality,Scripted,Sports,Talk Show,Variety
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [68]:
# importing more libraries
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier


In [70]:
features_train.shape

(796, 1038)

In [72]:
np.sqrt(796)

28.21347195933177

In [76]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

knn = KNeighborsClassifier(n_neighbors=28, weights='distance')

knn.fit(features_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=28, p=2,
           weights='distance')

In [77]:
cross_val_score(knn, features_train, y_train, cv=cv).mean()

0.50006329113924053

In [78]:
knn.score(features_test, y_test)

0.46783625730994149

In [79]:
dt = DecisionTreeClassifier(class_weight='balanced')
bdt = BaggingClassifier(DecisionTreeClassifier())
rf = RandomForestClassifier(class_weight='balanced', n_jobs=-1, n_estimators=20, random_state=42)
et = ExtraTreesClassifier(class_weight='balanced', n_jobs=-1)

In [80]:
dt.fit(features_train, y_train)
cross_val_score(dt, features_train, y_train, cv=cv).mean()

0.53156604998377155

In [81]:
dt.score(features_test, y_test)

0.54385964912280704

In [82]:
bdt.fit(features_train, y_train)
cross_val_score(bdt, features_train, y_train, cv=cv).mean()

0.56140863356053239

In [83]:
bdt.score(features_test, y_test)

0.61988304093567248

In [84]:
rf.fit(features_train, y_train)
cross_val_score(rf, features_train, y_train, cv=cv).mean()

0.55639281077572211

In [85]:
rf.score(features_test, y_test)

0.60818713450292394

In [89]:
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = features_train.columns,
                                    columns=['importance'])
feature_importances.sort_values(by='importance', ascending=False).head(40)

,importance
turn,0.009906
Crime,0.009536
Scripted,0.008463
Drama,0.007695
life,0.007027
family,0.006355
series,0.006253
friends,0.005755
days,0.005625
world,0.005507
